# Vibração Forçada NGL Via Impedância Mecânica

Este notebook resolve o problema de vibração forçada via a matriz de impedância mecânica.

Claramente este procedimento só é aplicável se todas as forças tiverem a mesma frequência e *em fase*.

A diferença entre este procedimento e o que fizemos para 2 GL é que aqui faremos todo o procedimento puramento numérico, sem o uso de matemática simbólica.

Este notebook pode ser usado para qualquer número de graus de liberdade que você tenha paciência de entrar com as matrizes "na mão", mas a plotagem está limitada a 10 GL.
## Equação de movimento

A equação de movimento é

$$ \boldsymbol{m}  \boldsymbol{\ddot x} + \boldsymbol{c} \boldsymbol{\dot x}+ \boldsymbol{k} \boldsymbol{x} = \boldsymbol{F_0}e^{i\omega t}.$$

Vamos supor que a resposta seja da forma $\boldsymbol{x}(t) = \boldsymbol{X}e^{i\omega t}$, onde $ \boldsymbol{X}$ é uma matriz coluna de números complexos e $\omega$ é a frequência da força de excitação.

Introduzindo esta solução na equação de movimento ficamos com

$$\left( - \omega^2 \boldsymbol{m} + i \omega \boldsymbol{c} + \boldsymbol{k} \right) \boldsymbol{X}e^{i\omega t} = \boldsymbol{F_0}e^{i\omega t},$$

É claro então que 

$$\left( \boldsymbol{k} - \omega^2 \boldsymbol{m} + i \omega \boldsymbol{c} \right) \boldsymbol{X} = \boldsymbol{F_0},$$

ou

$$  \boldsymbol{Z} \boldsymbol{X} = \boldsymbol{F_0},$$

com $\boldsymbol{Z} = \boldsymbol{k} -\omega^2 \boldsymbol{m} + i \omega \boldsymbol{c} $ sendo a matriz de impedância mecânica.

Claramente basta resolver este sistema de equações para a a resposta complexa $\boldsymbol{X}$ e depois calcular as amplitudes e fases.

## Frequências naturais e modos normais

Não é estritamente necessário para este tipo de problema, mas sempre é importante calcular as frequências naturais e modos normais do sistema. Vamos resolver o problema de autovalores e autovetores

$$ \lambda \boldsymbol{X} = \boldsymbol{\hat k}\boldsymbol{X},$$

com $\lambda = \omega^2$ e $\boldsymbol{\hat k} = \boldsymbol{m}^{-1} \boldsymbol{k}$.

## Setup

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as ipw

## Definição do sistema

O sistema é definido através das matrizes de massa, rigidez e amortecimento e da força externa.

Vamos chamar a frequẽncia da força externa aplicada de $\omega_0$, já que usaremos $\omega$ para as frequências naturais.

In [32]:
m = np.array([[1.0, 0.0, 0.0],
              [0.0, 1.0, 0.0],
              [0.0, 0.0, 1.0]], dtype=np.float64)

c = np.array([[ 2.4, -1.2,  0.0],
              [-1.2,  2.4, -1.2],
              [ 0.0, -1.2,  1.2]], dtype=np.float64)

k = np.array([[ 80.0, -40.0,   0.0],
              [-40.0,  80.0, -40.0],
              [  0.0, -40.0,  40.0]], dtype=np.float64)

F0 = np.array([10.0, 20.0, -30.0], dtype=np.float64)
ω0 = 7.0

## Frequências naturais e modos normais

Não vamos normalizar os modos normais pois não vamos usá-los para nada aqui.

In [33]:
λ, X = np.linalg.eig(np.linalg.inv(m)@k)
ω = np.sqrt(λ)
perm = np.argsort(ω)     # reordering index
ω = ω[perm]
X[...] = X[:, perm]      # modal matrix
display(ω)
display(X)

array([ 2.81469191,  7.88659149, 11.39645489])

array([[ 0.32798528, -0.73697623, -0.59100905],
       [ 0.59100905, -0.32798528,  0.73697623],
       [ 0.73697623,  0.59100905, -0.32798528]])

## Impedância mecânica

In [34]:
Ziw = k - ω0**2*m - 1j*ω0*c
display(Ziw)

array([[ 31.-16.8j, -40. +8.4j,   0. +0.j ],
       [-40. +8.4j,  31.-16.8j, -40. +8.4j],
       [  0. +0.j , -40. +8.4j,  -9. -8.4j]])

## Resposta

Vamos chamar a reposta $X_t$, já usamos $X$ para a matriz modal.

In [35]:
Xt = np.linalg.solve(Ziw, F0)
display(Xt)

array([ 0.82650276+0.84030013j,  0.65089971+0.44079038j,
       -0.65865489-0.73681741j])

### Amplitudes

In [36]:
Xa = np.abs(Xt)
display(Xa)

array([1.17864801, 0.78610852, 0.98829457])

### Fases

In [37]:
ϕ = np.angle(Xt)
display(ϕ)

array([ 0.79367572,  0.59526066, -2.30024144])

## Plotagem

In [38]:
τ = 2*np.pi/ω0
nτ = 5
npτ = 100
times = np.linspace(0, nτ*τ, nτ*npτ)

In [39]:
x = Xa[:,np.newaxis]*np.cos(ω0*times[np.newaxis,:] + ϕ[:, np.newaxis])

In [40]:
N = k.shape[0]
modes = range(1, min(N, 10)+1)
vxs = {f'x{i}' : ipw.Checkbox(value=True,description=f'$x_{i}$', disabled=False, indent=False) 
            for i in modes}
title = ipw.HTML('<h1> Resposta</h1><p> Escolha os modos para visualizar abaixo.</p>')

def make_plot(**args):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title('Deslocamento (m)', fontsize=18)
    ax.set_xlabel('Tempo (s)', fontsize=18)
    ax.set_ylabel('x(t)', fontsize=18)
    lines = [ ax.plot(times, x[i]) for i, c in enumerate(vxs.values()) if c.value ]
        
plot = ipw.interactive_output(make_plot, vxs)
bar = ipw.HBox(list(vxs.values()))
gui = ipw.VBox([title, bar, plot])
gui